# 1. Quickstart

Run the following to check if the container is running:

```$ docker ps --format "table {{.Image}}\t{{.Names}}\t{{.Status}}" | grep gai-ttt```

Once it is started, the status will look like this:

```gai-ttt:latest   gai-ttt   Up 2 minutes```

Otherwise, run `gai docker start`.



a) Patch OpenAI Client

In [1]:
from rich.console import Console
console = Console()

import os
# Replace this with your OpenAI API key if you want to use it
if os.environ.get("OPENAI_API_KEY") is None:
    os.environ["OPENAI_API_KEY"] = ""

# Load and patch openai
from openai import OpenAI
client = OpenAI()
from gai.ttt.client.completions import Completions

# get url from default client config
from gai.lib.config.config_utils import get_client_config

client = Completions.PatchOpenAI(client, get_client_config("ttt").url)


b) Create Completions

In [2]:
# Use openai
if os.environ.get("OPENAI_API_KEY"):
    console.print("[yellow]Use GPT-4o create:[/]")
    response=client.chat.completions.create(model="gpt-4o",messages=[{"role":"user","content":"Tell me a one sentence story"}])
    console.print(f"[white]{response}[/]")
    console.print("[white]extract:[/]")
    print(f"[white]{response.extract()}[/]")

# Use exllama
console.print("[bright_yellow]Use Exllama create:[/]")
response=client.chat.completions.create(model="exllamav2-mistral7b",messages=[{"role":"user","content":"Tell me a one sentence story"}])
console.print(f"[bright_white italic]{response}[/]")
console.print("[bright_white italic]extract:[/]")
console.print(f"[bright_white italic]{response.extract()}[/]")


Use Exllama create:

ERROR    completions._generate_dict: error='NoneType' object has no attribute 'startswith' response=None


AttributeError: 'NoneType' object has no attribute 'startswith'

c) Stream Completions

In [3]:
# Use openai
if os.environ.get("OPENAI_API_KEY"):
    console.print("[yellow]Use GPT-4o stream:[/]")
    response=client.chat.completions.create(model="gpt-4o",messages=[{"role":"user","content":"Tell me a one sentence story"}],stream=True)
    for chunk in response:
        chunk = chunk.extract()
        if chunk and type(chunk) is str:
            print(chunk,end="",flush=True)
print()
# Use exllama
console.print("[bright_yellow]Use Exllama stream:[/]")
response=client.chat.completions.create(model="exllamav2-mistral7b",messages=[{"role":"user","content":"Tell me a one sentence story"}],stream=True)
for chunk in response:
    chunk = chunk.extract()
    if chunk and type(chunk) is str:
            print(chunk,end="",flush=True)



Use Exllama stream:

 A young girl found a lost puppy and took it home to care for it

d) Create Tool Call

In [4]:
# Use openai
if os.environ.get("OPENAI_API_KEY"):
    console.print("[yellow]Use GPT-4o create tool call:[/]")
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": "What is the current time in Singapore?"}],
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "google",
                    "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "search_query": {
                                "type": "string",
                                "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                            }
                        },
                        "required": ["search_query"]
                    }
                }
            }
        ],
        tool_choice="required",
        stream=False
    )
    console.print(f"[bright_white italic]{json.dumps(response.extract(),indent=2)}[/]")
    print()

# Use exllama
console.print("[bright_yellow]Use Exllama create:[/]")
response = client.chat.completions.create(
    model="exllamav2-mistral7b",
    messages=[{"role": "user", "content": "What is the current time in Singapore?"},
              {"role": "assistant","content":""}],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }
    ],
    tool_choice="required",
    stream=False
)
import json
console.print(f"[bright_white italic]{json.dumps(response.extract(),indent=2)}[/]")


Use Exllama create:

{
  "type": "function",
  "name": "google",
  "arguments": "{\"search_query\": \"current time Singapore\"}"
}

e) JSON Mode

In [5]:
from pydantic import BaseModel
class Book(BaseModel):
    title: str
    summary: str
    author: str
    published_year: int
response_model = Book.schema()

data = """Foundation is a science fiction novel by American writer
        Isaac Asimov. It is the first published in his Foundation Trilogy (later
        expanded into the Foundation series). Foundation is a cycle of five
        interrelated short stories, first published as a single book by Gnome Press
        in 1951. Collectively they tell the early story of the Foundation,
        an institute founded by psychohistorian Hari Seldon to preserve the best
        of galactic civilization after the collapse of the Galactic Empire."""

# Use openai
if os.environ.get("OPENAI_API_KEY"):

    console.print("[yellow]Use GPT-4o stream:[/]")

    response = client.chat.completions.create(
        model="gpt-4o",
        tool_choice="none",
        messages=[{"role": "user", "content": f"Refer to the following content:\n\n{data}\n\nFormat it into JSON validated by this schema:{response_model}"}],
        stream=False
    )
    console.print(f"[bright_white italic]{json.dumps(response.extract(), indent=2)}[/]")
    print()

# Use exllama
console.print("[bright_yellow]Use Exllama create:[/]")
response = client.chat.completions.create(
    model="exllamav2-mistral7b",
    messages=[{"role": "user", "content": f"Refer to the following content:\n\n{data}\n\nFormat it into JSON validated by this schema:{response_model}"}],
    response_model=response_model,
    tool_choice="none",
    stream=False
)
import json
console.print(f"[bright_white italic]{json.dumps(response.extract(), indent=2)}[/]")


Use Exllama create:

{
  "type": "content",
  "content": " {\n    \"title\": \"Foundation\",\n    \"summary\": \"Foundation is a science fiction novel by 
American writer Isaac Asimov. It is the first published in his Foundation Trilogy (later expanded into the 
Foundation series). Foundation is a cycle of five interrelated short stories, first published as a single book by 
Gnome Press in 1951. Collectively they tell the early story of the Foundation, an institute founded by 
psychohistorian Hari Seldon to preserve the best of galactic civilization after the collapse of the Galactic 
Empire.\",\n    \"author\": \"Isaac Asimov\",\n    \"published_year\": 1951\n}"
}